In [1]:
import warnings
warnings.filterwarnings('ignore')


For these examples we are going to use the library [MIP](https://www.python-mip.com/)

In [2]:
import numpy as np

from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY

import folium

# The Travelling Salesman

Given a set of $n$ cities $C = {1, 2, \ldots, n}$ and a distance matrix $d_{i,j}$ representing the distance between cities $i$ and $j$, the TSP seeks to find the shortest Hamiltonian cycle (i.e., a cycle that visits each city exactly once) that starts and ends at a specified city (e.g., city 1).

Let $x_{i,j}$ be a binary variable that takes the value 1 if the TSP tour includes an edge from city $i$ to city $j$, and 0 otherwise. Let $u_i$ be a continuous variable that represents the position of city $i$ in the Hamiltonian cycle.

We can then formulate the TSP as the following MILP:

$$
\text{Minimize: } \sum_{i=1}^{n}\sum_{j=1}^{n} d_{i,j} x_{i,j}
$$

Subject to:

$$
\begin{aligned}
\sum_{j=1}^{n} x_{i,j} &= 1 && \forall i \in C \\
\sum_{i=1}^{n} x_{i,j} &= 1 && \forall j \in C \\
u_i - u_j + nx_{i,j} &\leq n-1 && \forall i,j \in C, i \neq 1, j \neq 1 \\
2 \leq u_i &\leq n && \forall i \in C \\
x_{i,j} &\in \{0,1\} && \forall i,j \in C \\
\end{aligned}
$$

The first constraint ensures that each city is visited exactly once. The second constraint ensures that the tour is a cycle that visits each city exactly once. The third constraint (known as the subtour elimination constraint) ensures that the tour is a Hamiltonian cycle by preventing the formation of subcycles. The fourth constraint ensures that the position variables are in the correct range. Finally, the fifth constraint ensures that the edge variables are binary.

In [3]:
cities = ["Santander", "Gijon", "Madrid", "Sevilla", "Barcelona", "Bilbao", "Valencia", "Vigo"]

In [4]:
# Define the coordinates for the cities
coordinates = {"Santander": [43.4623, -3.8],
               "Gijon": [43.5321, -5.6644],
               "Madrid": [40.4168, -3.7038],
               "Sevilla": [37.3891, -5.9845],
               "Barcelona": [41.3851, 2.1734],
               "Bilbao": [43.2630, -2.9340],
               "Valencia": [39.4699, -0.3763],
               "Vigo": [42.2406, -8.7207]}

In [5]:
# Define the distances between the cities
distances = [[0, 204, 426, 916, 724, 111, 605, 435],
             [204, 0, 429, 980, 944, 174, 690, 348],
             [426, 429, 0, 536, 620, 394, 353, 824],
             [916, 980, 536, 0, 995, 1063, 537, 741],
             [724, 944, 620, 995, 0, 612, 350, 1133],
             [111, 174, 394, 1063, 612, 0, 519, 602],
             [605, 690, 353, 537, 350, 519, 0, 688],
             [435, 348, 824, 741, 1133, 602, 688, 0]]

In [19]:
# Create a map object centered at the geographic center of Spain
map = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

# Add markers for each city to the map
for city, coords in coordinates.items():
    folium.Marker(location=coords, tooltip=city).add_to(map)

# Add lines between each pair of cities, with distance displayed as a popup
for i in range(len(cities)):
    for j in range(i+1, len(cities)):
        start = coordinates[cities[i]]
        end = coordinates[cities[j]]
        distance = distances[i][j]
        folium.PolyLine(locations=[start, end], tooltip=f"{distance} km", color = 'plum').add_to(map)

# Display the map
map

In [7]:
dists = []
p=1
for a in range(np.shape(distances)[0]):
    dists.append(distances[a][p:])
    p+=1
dists

[[204, 426, 916, 724, 111, 605, 435],
 [429, 980, 944, 174, 690, 348],
 [536, 620, 394, 353, 824],
 [995, 1063, 537, 741],
 [612, 350, 1133],
 [519, 602],
 [688],
 []]

In [8]:
# number of nodes and list of vertices
n, V = len(dists), set(range(len(dists)))

**Model name:**

In [9]:
name_model = 'Salesman'

We initialize the problem using the **Model** function from **MIP**

In [10]:
model = Model(name_model)

In [11]:
# distances matrix
c = [[0 if i == j
      else dists[i][j-i-1] if j > i
      else dists[j][i-j-1]
      for j in V] for i in V]

We add the different iterms to the problem

In [12]:
# binary variables indicating if arc (i,j) is used on the route or not
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]

# continuous variable to prevent subtours: each city will have a
# different sequential id in the planned route except the first one
y = [model.add_var() for i in V]

We give the model with the objective function to minimize

In [13]:
# objective function: minimize the distance
model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

We define the constraints

In [14]:
# constraint : leave each city only once
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# constraint : enter each city only once
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# subtour elimination
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

**We optimize the model**

In [15]:
# optimizing
model.optimize()

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 28 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 58 (0) rows, 63 (-9) columns and 238 (0) elements
Clp1000I sum of infeasibilities 3.45773e-05 - average 5.9616e-07, 37 fixed columns
Coin0506I Presolve 42 (-16) rows, 11 (-52) columns and 96 (-142) elements
Clp0029I End of values pass after 11 iterations
Clp0000I Optimal - objective value 2783.5556
Clp0000I Optimal - objective value 2783.5556
Coin0511I After Postsolve, objective 2783.5556, infeasibilities - dual 0 (0), primal 0 (0)
Clp0014I Perturbing problem by 0.001% of 0.95238197 - largest nonzero change 2.1344061e-05 ( 0.0011299797%) - largest zero change 1.7990915e-05
Clp0000I Optimal - objective value 2783.5556
Clp0000I Optimal - objective value 2783.5556
Clp0000I Optimal - objective value 2783.5556
Coin0511I After Postsolve, objective 2783.5556, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal 

<OptimizationStatus.OPTIMAL: 0>

**Extract results**

In [16]:
# checking if a solution was found
if model.num_solutions:
    out.write('route with total distance %g found: %s'
              % (model.objective_value, cities[0]))
    optimal_route, optimal_id = [], []
    nc = 0
    while True:
        nc = [i for i in V if x[nc][i].x >= 0.99][0]
        out.write(' -> %s' % cities[nc])
        optimal_route.append(cities[nc])
        optimal_id.append(nc)
        
        if nc == 0:
            break
    out.write('\n')

route with total distance 3255 found: Santander -> Bilbao -> Barcelona -> Valencia -> Madrid -> Sevilla -> Vigo -> Gijon -> Santander


In [17]:
optimal_route = np.append(cities[0], optimal_route)

In [23]:
# Create a map object centered at the geographic center of Spain
map = folium.Map(location=[40.4168, -3.7038], zoom_start=6)

# Add markers for each city to the map
for city, coords in coordinates.items():
    folium.Marker(location=coords, tooltip=city).add_to(map)

# Add lines between each pair of cities, with distance displayed as a popup
for i in range(len(optimal_route)-1):
    start = coordinates[optimal_route[i]]
    end = coordinates[optimal_route[i+1]]
    distance = distances[cities.index(optimal_route[i])][cities.index(optimal_route[i+1])]
    folium.PolyLine(locations=[start, end], tooltip=f"{distance} km").add_to(map)

# Add a layer to the map to show the optimal route
folium.PolyLine(locations=[coordinates[c] for c in optimal_route], color='darkmagenta', weight=5).add_to(map)

# Display the map
map
